## Bing Grounding Agent with Agent Framework

In [7]:

import asyncio
import os

from agent_framework import ChatAgent, HostedWebSearchTool
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

from openai import AzureOpenAI
from dotenv import load_dotenv 
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import os
from dotenv import load_dotenv
from typing import Annotated
from pydantic import Field

# Import for router hints support
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential


In [8]:
load_dotenv(override=True) # take environment variables from .env.

True

In [4]:
def write_to_file(model_name: str, question: str, response: str):
        try:
            print('write to csv file.')
            '''write to a csv file append a row to a column response and model name'''
            with open("bing_grounding_agent_responses.csv", "a") as f:
                f.write(f'"{model_name}","{question.replace(chr(10), " ").replace(chr(13), " ")}","{response.replace(chr(10), " ").replace(chr(13), " ")}"\n')
            print('✅ Successfully wrote to file')
        except Exception as e:
            print(f'❌ Error writing to file: {e}')


def ask_question_with_model_router(question: Annotated[str, Field(description="Ask Azure Model Router Question")]) -> str:
        print('🔍 DEBUG: ask_question_with_model_router FUNCTION WAS CALLED!')
        print('+++++++++++++++++++++++++++++++++++++++++++')
        print("QUESTION RECEIVED IN TOOL:")
        print(question)
        print('+++++++++++++++++++++++++++++++++++++++++++')
        
        try:
            # Use standard AzureOpenAI client
            endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
            api_key = os.getenv("AZURE_OPENAI_KEY")
            deployment_name = os.getenv("AZURE_OPENAI_MODEL")
            
            print(f"Using deployment: {deployment_name}")
            print(f"Endpoint: {endpoint}")
            
            client = AzureOpenAI(
                azure_endpoint=endpoint,
                api_key=api_key,
                api_version="2024-05-01-preview"
            )

            completion = client.chat.completions.create(
                model=deployment_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. You are asked complex questions. Think hard before answering. Use Reasoning to answer the question."},
                    {"role": "user", "content": question}
                ],
                max_tokens=30657
            )

            print("completion model used: " + completion.model)
            
            # Extract response text
            response_text = completion.choices[0].message.content
            print("About to write to file")
            write_to_file(completion.model, question, response_text)
            print("Returned from write_to_file")
            return f"content: {response_text}, model: {completion.model}"
        except Exception as e:
            print(f'❌ Error in ask_question_with_model_router: {e}')
            raise

    

# Create the agent
bing_search_tool = HostedWebSearchTool(
    name="BingGroundingSearch",
    description="Search the web for current information using Bing",
)

instructions = """

You are “Router Test Harness”.

MISSION
- When the user submits a question, DO NOT answer it.
- Rewrite the question into a higher-complexity “test prompt” designed to elicit a reasoning model from the Azure Model Router.
- Send ONLY the rewritten prompt (not the original) to the `run_router` tool.
- Ask for pros and cons, ask for companison analysis, ask for step by step reasoning, or ask for a plan with steps to solve the problem.


PIPELINE
(1) REWRITE
- Create a single-turn test prompt that forces: multi-step planning, constraints, verification, and synthesis.
- Add domain scaffolding (planning, constraints, success criteria, intermediate representation).
- Keep it concise (< 500 tokens) and self-contained. Do NOT include conversation history.
- Use words like: "think harder", "plan", "steps", "constraints", "verify", "synthesize", "criteria", "representation".
- flagging issues and challenging assumptions
- You should not be fast, you should take your time before responding.

(2) ROUTE (TOOL: run_router)
- Input: the rewritten prompt only.
- If a reasoning model was NOT selected, perform up to 2 escalations (see below), each time rewriting to increase complexity, then call `run_router` again. Never exceed 3 runs total.
"""
       
client = AzureAIAgentClient(async_credential=AzureCliCredential())
agent = ChatAgent(
    chat_client=client,
    name="BingGroundingSearchAgent",
    instructions=(
        instructions
    ),
    tools=[ ask_question_with_model_router]
)

print("✅ Agent created and ready to use!")

✅ Agent created and ready to use!


In [5]:
async def ask_agent(user_input: str):
    """Ask the agent a question and get a response"""
    print("=== Azure AI Agent with Bing Grounding Search ===\n")
    print(f"User: {user_input}")

    print("🔍 DEBUG: About to call agent.run with tool_choice")
    response_text = await agent.run(user_input, tool_choice=ask_question_with_model_router)
    print(f"🔍 DEBUG: Agent returned. Response type: {type(response_text)}")
    print(f"Agent: {response_text.text}\n")
    return response_text

In [9]:
for _ in range(100):
    print(_)
    await ask_agent("Give me a SQL Server 2022 scripts to create a datawarehouse for the purchase department, and provide code explanations, generate  seed script, and calculate metrics for seed data following the EBITA  model.  Using the generated seed script, analyze the financial statements for the last 3 years, identify discrepancies, and proces a five-step reasoning report with citations from teh annexed documents using formal rigor")

0
=== Azure AI Agent with Bing Grounding Search ===

User: Give me a SQL Server 2022 scripts to create a datawarehouse for the purchase department, and provide code explanations, generate  seed script, and calculate metrics for seed data following the EBITA  model.  Using the generated seed script, analyze the financial statements for the last 3 years, identify discrepancies, and proces a five-step reasoning report with citations from teh annexed documents using formal rigor
🔍 DEBUG: About to call agent.run with tool_choice
🔍 DEBUG: ask_question_with_model_router FUNCTION WAS CALLED!
+++++++++++++++++++++++++++++++++++++++++++
QUESTION RECEIVED IN TOOL:
Develop a formal multi-step analysis of constructing a data warehouse for a purchase department using SQL Server 2022. Begin by planning and describing the key entities and tables relevant for purchase and financial analysis according to EBITA (Earnings Before Interest, Taxes, and Amortization) model constraints. Next, write scripts to 

CancelledError: 